In [1]:
import numpy as np
import pandas as pd
import fire
import matplotlib.pyplot as plt
import torch
import torch.optim.swa_utils as swa_utils
import torchcde
import torchsde
import tqdm
import funcions
from funcions import Generator, Discriminator
import json

In [2]:
demographic = pd.read_csv('demographic.csv')
labs = pd.read_csv('labs.csv')

*Next step:* Hem de triar, pel dataset de demographic i pel de labs, algunes de les columnes. Pel de demographic, a vista, i pel de labs, les de l'excel amb les components d'un blood test.

In [3]:
demographic

,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,...,DMDHREDU,DMDHRMAR,DMDHSEDU,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR
0,73557,8,2,1,69,NaN,4,4,1.0,NaN,...,3.0,4.0,NaN,13281.237386,13481.042095,1,112,4.0,4.0,0.84
1,73558,8,2,1,54,NaN,3,3,1.0,NaN,...,3.0,1.0,1.0,23682.057386,24471.769625,1,108,7.0,7.0,1.78
2,73559,8,2,1,72,NaN,3,3,2.0,NaN,...,4.0,1.0,3.0,57214.803319,57193.285376,1,109,10.0,10.0,4.51
3,73560,8,2,1,9,NaN,3,3,1.0,119.0,...,3.0,1.0,4.0,55201.178592,55766.512438,2,109,9.0,9.0,2.52
4,73561,8,2,2,73,NaN,3,3,1.0,NaN,...,5.0,1.0,5.0,63709.667069,65541.871229,2,116,15.0,15.0,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10170,83727,8,2,1,26,NaN,2,2,2.0,NaN,...,3.0,NaN,NaN,27141.631824,28115.568221,1,104,77.0,77.0,NaN
10171,83728,8,2,2,2,24.0,1,1,2.0,24.0,...,3.0,1.0,3.0,15274.475360,16512.138781,1,107,8.0,8.0,1.95
10172,83729,8,2,2,42,NaN,4,4,2.0,NaN,...,5.0,3.0,NaN,24122.248900,26902.344381,1,104,7.0,7.0,3.66
10173,83730,8,2,1,7,NaN,2,2,1.0,84.0,...,4.0,1.0,3.0,25521.878546,26686.025573,2,109,6.0,6.0,1.05


In [4]:
demographic = demographic[['SEQN','DMDBORN4','RIDAGEYR','RIAGENDR','RIDEXPRG','RIDRETH3']]
demographic

,SEQN,DMDBORN4,RIDAGEYR,RIAGENDR,RIDEXPRG,RIDRETH3
0,73557,1,69,1,NaN,4
1,73558,1,54,1,NaN,3
2,73559,1,72,1,NaN,3
3,73560,1,9,1,NaN,3
4,73561,1,73,2,NaN,3
...,...,...,...,...,...,...
10170,83727,1,26,1,NaN,2
10171,83728,1,2,2,NaN,1
10172,83729,2,42,2,2.0,4
10173,83730,1,7,1,NaN,2


In [5]:
demographic

,SEQN,DMDBORN4,RIDAGEYR,RIAGENDR,RIDEXPRG,RIDRETH3
0,73557,1,69,1,NaN,4
1,73558,1,54,1,NaN,3
2,73559,1,72,1,NaN,3
3,73560,1,9,1,NaN,3
4,73561,1,73,2,NaN,3
...,...,...,...,...,...,...
10170,83727,1,26,1,NaN,2
10171,83728,1,2,2,NaN,1
10172,83729,2,42,2,2.0,4
10173,83730,1,7,1,NaN,2


In [6]:
demographic.rename(columns={'SEQN':'id','DMDBORN4':'country','RIDAGEYR':'age_years','RIAGENDR':'gender',
                    'RIDEXPRG':'pregnancy_status','RIDRETH3':'race'},inplace=True)

C:\Users\alex.padros\AppData\Local\Temp\4\ipykernel_16552\357610968.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demographic.rename(columns={'SEQN':'id','DMDBORN4':'country','RIDAGEYR':'age_years','RIAGENDR':'gender',


In [7]:
demographic.isnull().sum()

id                     0
country                0
age_years              0
gender                 0
pregnancy_status    8866
race                   0
dtype: int64

In [8]:
demographic.dtypes

id                    int64
country               int64
age_years             int64
gender                int64
pregnancy_status    float64
race                  int64
dtype: object

In [9]:
demographic['pregnancy_status']=demographic['pregnancy_status'].fillna(0)
demographic['pregnancy_status']=demographic['pregnancy_status'].astype(np.int64)

C:\Users\alex.padros\AppData\Local\Temp\4\ipykernel_16552\1475092732.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demographic['pregnancy_status']=demographic['pregnancy_status'].fillna(0)
C:\Users\alex.padros\AppData\Local\Temp\4\ipykernel_16552\1475092732.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demographic['pregnancy_status']=demographic['pregnancy_status'].astype(np.int64)


In [10]:
labs = pd.read_csv('labs.csv')

In [11]:
labs.isnull().sum()

SEQN           0
URXUMA      1761
URXUMS      1761
URXUCR.x    1761
URXCRS      1761
            ... 
URDUURLC    7149
URXPREG     8552
URXUAS      7151
LBDB12      4497
LBDB12SI    4497
Length: 424, dtype: int64

In [12]:
labs = labs[['SEQN','LBDHDDSI','LBDTCSI',
      'LBXSUA','LBXSATSI','LBXSASSI','LBDSTPSI','LBDSALSI','LBXSTB','LBXHGB','LBXHCT',
      'LBXRBCSI','LBXMCVSI','LBXMCHSI','LBXMC','LBXWBCSI','LBDNENO','LBDLYMNO','LBDMONO',
      'LBDEONO','LBDBANO','LBDSIRSI','LBXPLTSI']]
labs

,SEQN,LBDHDDSI,LBDTCSI,LBXSUA,LBXSATSI,LBXSASSI,LBDSTPSI,LBDSALSI,LBXSTB,LBXHGB,...,LBXMCHSI,LBXMC,LBXWBCSI,LBDNENO,LBDLYMNO,LBDMONO,LBDEONO,LBDBANO,LBDSIRSI,LBXPLTSI
0,73557,1.68,4.32,3.3,16.0,16.0,65.0,41.0,0.8,15.2,...,29.9,33.4,4.7,2.0,2.0,0.5,0.2,0.1,10.4,204.0
1,73558,1.29,4.40,4.7,29.0,18.0,78.0,47.0,0.9,11.9,...,31.0,32.5,12.6,7.4,3.4,1.0,0.8,0.1,14.1,314.0
2,73559,1.55,3.26,5.7,16.0,22.0,59.0,37.0,0.6,17.2,...,31.1,34.3,7.2,4.9,1.0,0.8,0.4,0.1,17.6,237.0
3,73560,1.58,4.34,NaN,NaN,NaN,NaN,NaN,NaN,12.9,...,28.0,34.0,7.8,4.6,2.3,0.7,0.1,0.0,NaN,240.0
4,73561,2.20,5.20,4.2,28.0,36.0,71.0,43.0,0.5,14.5,...,30.6,33.0,6.6,4.5,1.4,0.5,0.2,0.1,16.3,300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9808,83727,1.42,4.91,5.3,26.0,27.0,75.0,49.0,0.9,14.9,...,29.3,32.9,5.1,2.9,1.6,0.3,0.2,0.0,13.1,226.0
9809,83728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.2,...,28.9,35.0,6.6,1.4,4.4,0.5,0.2,0.0,NaN,441.0
9810,83729,1.24,3.93,5.3,13.0,26.0,67.0,41.0,0.2,12.2,...,29.2,32.8,6.4,3.2,2.5,0.6,0.1,0.0,8.8,213.0
9811,83730,1.32,4.32,NaN,NaN,NaN,NaN,NaN,NaN,13.7,...,27.2,33.7,9.9,4.0,5.0,0.6,0.3,0.0,NaN,352.0


In [13]:
labs.isnull().sum()

SEQN           0
LBDHDDSI    2189
LBDTCSI     2189
LBXSUA      3262
LBXSATSI    3262
LBXSASSI    3262
LBDSTPSI    3269
LBDSALSI    3260
LBXSTB      3264
LBXHGB      1269
LBXHCT      1269
LBXRBCSI    1269
LBXMCVSI    1269
LBXMCHSI    1269
LBXMC       1269
LBXWBCSI    1269
LBDNENO     1294
LBDLYMNO    1294
LBDMONO     1294
LBDEONO     1294
LBDBANO     1294
LBDSIRSI    3286
LBXPLTSI    1269
dtype: int64

In [14]:
# This will keep only rows which have nan's less than 6 in the dataframe, 
# and will remove all having nan's > 6.
labs_clean = labs[labs.isnull().sum(axis=1) <= 6]
labs_clean.isnull().sum()

SEQN         0
LBDHDDSI     1
LBDTCSI      1
LBXSUA       2
LBXSATSI     2
LBXSASSI     2
LBDSTPSI     8
LBDSALSI     0
LBXSTB       4
LBXHGB       0
LBXHCT       0
LBXRBCSI     0
LBXMCVSI     0
LBXMCHSI     0
LBXMC        0
LBXWBCSI     0
LBDNENO     19
LBDLYMNO    19
LBDMONO     19
LBDEONO     19
LBDBANO     19
LBDSIRSI    26
LBXPLTSI     0
dtype: int64

In [15]:
for col in labs_clean.columns:
    labs_clean[col] = labs_clean[col].fillna(labs_clean[col].mean())

C:\Users\alex.padros\AppData\Local\Temp\4\ipykernel_16552\119071111.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labs_clean[col] = labs_clean[col].fillna(labs_clean[col].mean())


In [16]:
labs_clean

,SEQN,LBDHDDSI,LBDTCSI,LBXSUA,LBXSATSI,LBXSASSI,LBDSTPSI,LBDSALSI,LBXSTB,LBXHGB,...,LBXMCHSI,LBXMC,LBXWBCSI,LBDNENO,LBDLYMNO,LBDMONO,LBDEONO,LBDBANO,LBDSIRSI,LBXPLTSI
0,73557,1.68,4.32,3.3,16.0,16.0,65.0,41.0,0.8,15.2,...,29.9,33.4,4.7,2.0,2.0,0.5,0.2,0.1,10.4,204.0
1,73558,1.29,4.40,4.7,29.0,18.0,78.0,47.0,0.9,11.9,...,31.0,32.5,12.6,7.4,3.4,1.0,0.8,0.1,14.1,314.0
2,73559,1.55,3.26,5.7,16.0,22.0,59.0,37.0,0.6,17.2,...,31.1,34.3,7.2,4.9,1.0,0.8,0.4,0.1,17.6,237.0
4,73561,2.20,5.20,4.2,28.0,36.0,71.0,43.0,0.5,14.5,...,30.6,33.0,6.6,4.5,1.4,0.5,0.2,0.1,16.3,300.0
5,73562,0.98,5.84,9.1,16.0,24.0,73.0,43.0,0.5,14.2,...,28.8,34.2,9.4,6.5,1.6,0.6,0.6,0.1,9.3,249.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9802,83721,1.42,3.57,5.7,47.0,24.0,70.0,44.0,0.7,14.4,...,32.2,34.4,6.8,4.4,1.4,0.6,0.3,0.1,16.1,152.0
9804,83723,1.27,4.81,4.2,23.0,24.0,78.0,41.0,0.7,14.3,...,35.2,35.5,7.7,3.9,2.9,0.8,0.1,0.0,15.8,181.0
9805,83724,1.32,4.06,6.1,12.0,20.0,60.0,38.0,0.5,11.8,...,31.8,33.9,6.1,4.2,1.2,0.6,0.1,0.1,15.0,180.0
9808,83727,1.42,4.91,5.3,26.0,27.0,75.0,49.0,0.9,14.9,...,29.3,32.9,5.1,2.9,1.6,0.3,0.2,0.0,13.1,226.0


In [17]:
labs_clean.iloc[:,1:]

,LBDHDDSI,LBDTCSI,LBXSUA,LBXSATSI,LBXSASSI,LBDSTPSI,LBDSALSI,LBXSTB,LBXHGB,LBXHCT,...,LBXMCHSI,LBXMC,LBXWBCSI,LBDNENO,LBDLYMNO,LBDMONO,LBDEONO,LBDBANO,LBDSIRSI,LBXPLTSI
0,1.68,4.32,3.3,16.0,16.0,65.0,41.0,0.8,15.2,45.4,...,29.9,33.4,4.7,2.0,2.0,0.5,0.2,0.1,10.4,204.0
1,1.29,4.40,4.7,29.0,18.0,78.0,47.0,0.9,11.9,36.7,...,31.0,32.5,12.6,7.4,3.4,1.0,0.8,0.1,14.1,314.0
2,1.55,3.26,5.7,16.0,22.0,59.0,37.0,0.6,17.2,49.9,...,31.1,34.3,7.2,4.9,1.0,0.8,0.4,0.1,17.6,237.0
4,2.20,5.20,4.2,28.0,36.0,71.0,43.0,0.5,14.5,43.8,...,30.6,33.0,6.6,4.5,1.4,0.5,0.2,0.1,16.3,300.0
5,0.98,5.84,9.1,16.0,24.0,73.0,43.0,0.5,14.2,41.5,...,28.8,34.2,9.4,6.5,1.6,0.6,0.6,0.1,9.3,249.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9802,1.42,3.57,5.7,47.0,24.0,70.0,44.0,0.7,14.4,41.9,...,32.2,34.4,6.8,4.4,1.4,0.6,0.3,0.1,16.1,152.0
9804,1.27,4.81,4.2,23.0,24.0,78.0,41.0,0.7,14.3,40.1,...,35.2,35.5,7.7,3.9,2.9,0.8,0.1,0.0,15.8,181.0
9805,1.32,4.06,6.1,12.0,20.0,60.0,38.0,0.5,11.8,34.8,...,31.8,33.9,6.1,4.2,1.2,0.6,0.1,0.1,15.0,180.0
9808,1.42,4.91,5.3,26.0,27.0,75.0,49.0,0.9,14.9,45.1,...,29.3,32.9,5.1,2.9,1.6,0.3,0.2,0.0,13.1,226.0


In [18]:
X = labs_clean.iloc[:,1:].values
X

array([[1.68e+00, 4.32e+00, 3.30e+00, ..., 1.00e-01, 1.04e+01, 2.04e+02],
       [1.29e+00, 4.40e+00, 4.70e+00, ..., 1.00e-01, 1.41e+01, 3.14e+02],
       [1.55e+00, 3.26e+00, 5.70e+00, ..., 1.00e-01, 1.76e+01, 2.37e+02],
       ...,
       [1.32e+00, 4.06e+00, 6.10e+00, ..., 1.00e-01, 1.50e+01, 1.80e+02],
       [1.42e+00, 4.91e+00, 5.30e+00, ..., 0.00e+00, 1.31e+01, 2.26e+02],
       [1.24e+00, 3.93e+00, 5.30e+00, ..., 0.00e+00, 8.80e+00, 2.13e+02]])

# MAIN

In [19]:
with open('./Params/model3.json') as f:
    params = json.load(f)
initial_noise_size = params['initial_noise_size'] # How many noise dimensions to sample at the start of the SDE.
noise_size = params['noise_size'] # How many dimensions the Brownian motion has.
hidden_size = params['hidden_size'] # How big the hidden size of the generator SDE and the discriminator CDE are.
mlp_size = params['mlp_size'] # How big the layers in the various MLPs are.
num_layers = params['num_layers'] # How many hidden layers to have in the various MLPs.
# Training hyperparameters. Be prepared to tune these very carefully, as with any GAN.
generator_lr = params['generator_lr'] # Learning rate often needs careful tuning to the problem.
discriminator_lr = params['discriminator_lr'] # Learning rate often needs careful tuning to the problem.
batch_size = params['batch_size'] # Batch size.
steps = params['steps'] # How many steps to train both generator and discriminator for.
init_mult1 = params['init_mult1'] # Changing the initial parameter size can help.
init_mult2 = params['init_mult2']
weight_decay = params['weight_decay'] # Weight decay.
swa_step_start = params['swa_step_start'] # When to start using stochastic weight averaging.
steps_per_print = params['steps_per_print'] # How often to print the loss.
t_size = params['t_size'] 
print(params)

{'initial_noise_size': 15, 'noise_size': 10, 'hidden_size': 22, 'mlp_size': 32, 'num_layers': 2, 'generator_lr': 0.01, 'discriminator_lr': 0.01, 'batch_size': 128, 'steps': 3000, 'init_mult1': 3, 'init_mult2': 0.5, 'weight_decay': 0.01, 'swa_step_start': 500, 'steps_per_print': 10, 't_size': 10}


In [20]:
# plotting hyperparameters
# num_plot_samples=50                  # How many samples to use on the plots at the end.
# plot_locs=(0.1, 0.3, 0.5, 0.7, 0.9)  # Plot some marginal distributions at this proportion of the way along.

In [21]:
is_cuda = torch.cuda.is_available()
device = 'cuda' if is_cuda else 'cpu'
if not is_cuda:
    print("Warning: CUDA not available; falling back to CPU but this is likely to be very slow.")

ts = torch.linspace(0, t_size - 1, t_size, device=device)
data_size = X.shape[1]

In [22]:
# dataloader
ys_coeffs = torch.from_numpy(X)
ys_coeffs = ys_coeffs.to(torch.float32)
ys_coeffs = torch.reshape(ys_coeffs,(X.shape[0],1,X.shape[1]))
dataset = torch.utils.data.TensorDataset(ys_coeffs)
train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [23]:
infinite_train_dataloader = (elem for it in iter(lambda: train_dataloader, None) for elem in it)

In [24]:
# Models
generator = Generator(data_size, initial_noise_size, noise_size, hidden_size, mlp_size, num_layers).to(device)
discriminator = Discriminator(data_size, hidden_size, mlp_size, num_layers).to(device)

In [25]:
generator

Generator(
  (_initial): MLP(
    (_model): Sequential(
      (0): Linear(in_features=15, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=22, bias=True)
    )
  )
  (_func): GeneratorFunc(
    (_drift): MLP(
      (_model): Sequential(
        (0): Linear(in_features=23, out_features=32, bias=True)
        (1): ReLU()
        (2): Linear(in_features=32, out_features=32, bias=True)
        (3): ReLU()
        (4): Linear(in_features=32, out_features=22, bias=True)
        (5): Tanh()
      )
    )
    (_diffusion): MLP(
      (_model): Sequential(
        (0): Linear(in_features=23, out_features=32, bias=True)
        (1): ReLU()
        (2): Linear(in_features=32, out_features=32, bias=True)
        (3): ReLU()
        (4): Linear(in_features=32, out_features=220, bias=True)
        (5): Tanh()
      )
    )
  )
  (_readout): Linear(in_features=22, out_features=22, bias=True)
)

In [26]:
discriminator

Discriminator(
  (_initial): MLP(
    (_model): Sequential(
      (0): Linear(in_features=22, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=32, bias=True)
      (3): ReLU()
      (4): Linear(in_features=32, out_features=22, bias=True)
    )
  )
  (_readout): Linear(in_features=22, out_features=1, bias=True)
)

In [27]:
# Weight averaging really helps with GAN training.
averaged_generator = swa_utils.AveragedModel(generator)
averaged_discriminator = swa_utils.AveragedModel(discriminator)

In [28]:
# Picking a good initialisation is important!
# In this case these were picked by making the parameters for the t=0 part of the generator be roughly the right
# size that the untrained t=0 distribution has a similar variance to the t=0 data distribution.
# Then the func parameters were adjusted so that the t>0 distribution looked like it had about the right variance.
# What we're doing here is very crude -- one can definitely imagine smarter ways of doing things.
# (e.g. pretraining the t=0 distribution)
with torch.no_grad():
    for param in generator._initial.parameters():
        param *= init_mult1
    for param in generator._func.parameters():
        param *= init_mult2


In [29]:
# Optimisers. Adadelta turns out to be a much better choice than SGD or Adam, interestingly.
generator_optimiser = torch.optim.Adadelta(generator.parameters(), lr=generator_lr, weight_decay=weight_decay)
discriminator_optimiser = torch.optim.Adadelta(discriminator.parameters(), lr=discriminator_lr,
                                               weight_decay=weight_decay)

In [30]:
def evaluate_loss(ts, batch_size, dataloader, generator, discriminator):
    with torch.no_grad():
        total_samples = 0
        total_loss = 0
        for real_samples, in dataloader:
            generated_samples = generator(ts, batch_size)
            generated_score = discriminator(generated_samples)
            real_score = discriminator(real_samples)
            loss = generated_score - real_score
            total_samples += batch_size
            total_loss += loss.item() * batch_size
    return total_loss / total_samples

In [31]:
# Train both generator and discriminator.
trange = tqdm.tqdm(range(steps))
for step in trange:
    real_samples, = next(infinite_train_dataloader)

    generated_samples = generator(ts, batch_size)
#     print(generated_samples.shape)
#     print(generated_samples)
    generated_score = discriminator(generated_samples)
    real_score = discriminator(real_samples)
    loss = generated_score - real_score
#     loss.backward()
    loss.backward()
    for param in generator.parameters():
        param.grad *= -1
    generator_optimiser.step()
    discriminator_optimiser.step()
    generator_optimiser.zero_grad()
    discriminator_optimiser.zero_grad()

    ###################
    # We constrain the Lipschitz constant of the discriminator using carefully-chosen clipping (and the use of
    # LipSwish activation functions).
    ###################
    with torch.no_grad():
        for module in discriminator.modules():
            if isinstance(module, torch.nn.Linear):
                lim = 1 / module.out_features
                module.weight.clamp_(-lim, lim)

    # Stochastic weight averaging typically improves performance.
    if step > swa_step_start:
        averaged_generator.update_parameters(generator)
        averaged_discriminator.update_parameters(discriminator)

    if (step % steps_per_print) == 0 or step == steps - 1:
        total_unaveraged_loss = evaluate_loss(ts, batch_size, train_dataloader, generator, discriminator)
        if step > swa_step_start:
            total_averaged_loss = evaluate_loss(ts, batch_size, train_dataloader, averaged_generator.module,
                                                averaged_discriminator.module)
            trange.write(f"Step: {step:3} Loss (unaveraged): {total_unaveraged_loss:.4f} "
                         f"Loss (averaged): {total_averaged_loss:.4f}")
        else:
            trange.write(f"Step: {step:3} Loss (unaveraged): {total_unaveraged_loss:.4f}")
generator.load_state_dict(averaged_generator.module.state_dict())
discriminator.load_state_dict(averaged_discriminator.module.state_dict())

# _, _, test_dataloader = get_data(batch_size=batch_size, device=device)

Step:   0 Loss (unaveraged): -0.0609


Step:  10 Loss (unaveraged): -0.0654


Step:  20 Loss (unaveraged): -0.0709


Step:  30 Loss (unaveraged): -0.0775


Step:  40 Loss (unaveraged): -0.0854


...

Step: 2980 Loss (unaveraged): -0.1055 Loss (averaged): -0.4261


Step: 2990 Loss (unaveraged): -0.1180 Loss (averaged): -0.4235


Step: 2999 Loss (unaveraged): -0.0937 Loss (averaged): -0.4331


100%|██████████████████████████████████████████████████████████████████████████████| 3000/3000 [22:25<00:00,  2.23it/s]


<All keys matched successfully>

In [59]:
prova=generator(ts, batch_size=1)
prova.shape
prova=prova[0]
torch.abs(prova[-1])

tensor([  7.6801,  27.4081,  23.6412,  37.2035,  53.4925, 103.3308,  55.6306,
         18.0615,  39.7406,  53.1277,  27.0966, 130.4899,  50.0291,  53.2631,
         23.7175,  19.1419,  10.8454,  10.4972,  15.5098,  13.9938,  42.0859,
        403.3273], grad_fn=<AbsBackward0>)

In [33]:
labs_clean.iloc[:,1:].describe()

,LBDHDDSI,LBDTCSI,LBXSUA,LBXSATSI,LBXSASSI,LBDSTPSI,LBDSALSI,LBXSTB,LBXHGB,LBXHCT,...,LBXMCHSI,LBXMC,LBXWBCSI,LBDNENO,LBDLYMNO,LBDMONO,LBDEONO,LBDBANO,LBDSIRSI,LBXPLTSI
count,6547.000000,6547.000000,6547.00000,6547.000000,6547.000000,6547.000000,6547.000000,6547.000000,6547.000000,6547.000000,...,6547.000000,6547.000000,6547.000000,6547.000000,6547.000000,6547.000000,6547.000000,6547.000000,6547.000000,6547.000000
mean,1.362269,4.729313,5.34984,24.068144,25.112758,71.077810,42.822514,0.639306,13.960883,41.284924,...,29.953093,33.785215,7.293111,4.272518,2.191575,0.581158,0.202773,0.044838,14.949778,237.965175
std,0.401247,1.076347,1.40365,18.486176,17.743534,4.679562,3.435935,0.308692,1.507656,4.081439,...,2.489708,1.072428,2.301252,1.793620,0.921684,0.193684,0.183908,0.052067,6.507336,59.704024
min,0.260000,1.780000,0.70000,6.000000,9.000000,47.000000,24.000000,0.100000,6.400000,17.900000,...,16.800000,28.000000,2.300000,0.400000,0.400000,0.100000,0.000000,0.000000,0.900000,18.000000
25%,1.090000,3.960000,4.30000,15.000000,19.000000,68.000000,41.000000,0.400000,13.000000,38.600000,...,28.800000,33.200000,5.700000,3.100000,1.700000,0.400000,0.100000,0.000000,10.600000,198.000000
50%,1.290000,4.630000,5.20000,20.000000,22.000000,71.000000,43.000000,0.600000,13.900000,41.300000,...,30.200000,33.800000,7.000000,4.000000,2.100000,0.600000,0.200000,0.000000,14.100000,232.000000
75%,1.580000,5.380000,6.20000,27.000000,27.000000,74.000000,45.000000,0.800000,15.000000,44.100000,...,31.500000,34.400000,8.400000,5.100000,2.600000,0.700000,0.202773,0.100000,18.400000,272.000000
max,4.470000,21.020000,13.30000,536.000000,882.000000,102.000000,56.000000,7.100000,19.500000,56.500000,...,74.500000,69.600000,55.700000,25.600000,49.000000,3.400000,4.300000,0.800000,99.800000,669.000000


In [ ]:
torch.save(generator, './Models/Model_3/generator')
torch.save(discriminator, './Models/Model_3/discriminator')

In [35]:
# generator = Generator()
# generator.load_state_dict(torch.load('./Models/Model_2/generator'))
# generator.eval()

In [36]:
# prova=generator(ts, batch_size=1)
# prova.shape
# prova=prova[0]
# torch.abs(prova[-1])